In [23]:
import pandas as pd
from plotly import graph_objects as go
import nevergrad as ng
from summer2 import AgeStratification, Overwrite

from estival.wrappers.nevergrad import optimize_model
import estival.priors as esp
import estival.targets as est
from estival.model import BayesianCompartmentalModel
from estival.wrappers import pymc as epm
from summer2 import CompartmentalModel
from summer2.parameters import Parameter, Function, DerivedOutput
from summer2.functions.time import get_sigmoidal_interpolation_function

from tb_incubator.demographics import add_extra_crude_birth_flow
from tb_incubator.constants import set_project_base_path
from tb_incubator.input import get_birth_rate, get_pop_death_data, get_death_rates

pd.options.plotting.backend = "plotly"
project_paths = set_project_base_path("../tb_incubator/")

In [24]:
# load birth data
birth_rates = get_birth_rate()

/Users/tys/Documents/research/2024/TB Model/incubator2024/tb_incubator/input.py:13: DtypeWarning:

Columns (2,3,4,7) have mixed types. Specify dtype option on import or set low_memory=False.



In [25]:
# load age-stratified, population & deaths data
pop_death = get_pop_death_data()

target_pops = pop_death.groupby(level=[0]).sum()["population"]

/Users/tys/Documents/research/2024/TB Model/incubator2024/tb_incubator/input.py:32: DtypeWarning:

Columns (3,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111) have mixed types. Specify dtype option on import or set low_memory=False.

/Users/tys/Documents/research/2024/TB Model/incubator2024/tb_incubator/input.py:68: DtypeWarning:

Columns (3,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111) have mixed types. Specify dtype option on import or set low_memory=False.

In [26]:
# load death rates
death_rates = get_death_rates()

/Users/tys/Documents/research/2024/TB Model/incubator2024/tb_incubator/input.py:32: DtypeWarning:

Columns (3,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111) have mixed types. Specify dtype option on import or set low_memory=False.

/Users/tys/Documents/research/2024/TB Model/incubator2024/tb_incubator/input.py:68: DtypeWarning:

Columns (3,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111) have mixed types. Specify dtype option on import or set low_memory=False.

In [27]:
death_rates.plot()

In [28]:
# Population per year
total_population_per_year = pop_death.groupby('year')['population'].sum()

In [29]:
# Arbitrary base model construction
model_comps = ["susceptible", "early latent", "late latent", "infectious", "recovered"]
model_times = [1850.0, 2024.0]
model = CompartmentalModel(
    times=model_times,
    compartments=model_comps,
    infectious_compartments=["infectious"],
)
init_pops = {"susceptible": 1e6, "infectious": 0.0}
model.set_initial_population(init_pops)

In [30]:
# TB transitions, some meaningless TB-related flows
#model.add_death_flow("TB death", Parameter("death rate"), "infectious")

In [31]:
# Demographic transitions
model.add_universal_death_flows("population_death", 1.0)
model.add_replacement_birth_flow("replacement_birth", "susceptible")


In [32]:
total_population_change = []
total_population_change = total_population_per_year.diff()
total_population_change = total_population_change.dropna()
total_population_change = total_population_change.drop(total_population_change.index[-1])
total_population_change.index = pd.to_numeric(total_population_change.index, errors='coerce')
total_population_change = pd.to_numeric(total_population_change, errors='coerce')

In [33]:
entry_rate = get_sigmoidal_interpolation_function(
    total_population_change.index, 
    total_population_change
)

In [34]:
# Arbitrary epidemiological parameter
params = {"starting population" : 1e6}

In [35]:
# Apply age stratification with age-specific death rate functions of time
agegroup_request = [[0, 4], [5, 14], [15, 34], [35, 49], [50, 100]]
age_strata = [i[0] for i in agegroup_request]
strat = AgeStratification("age", age_strata, model_comps)
death_adjs = {}
for age in age_strata:
    years = death_rates.index
    rates = death_rates[age]
    pop_death_func = get_sigmoidal_interpolation_function(years, rates)
    death_adjs[str(age)] = Overwrite(pop_death_func)
strat.set_flow_adjustments("population_death", death_adjs)
model.stratify_with(strat)

In [36]:
model.add_importation_flow("births", entry_rate, dest="susceptible", split_imports=True, dest_strata={"age": "0"})

In [37]:
# Track population
model.request_output_for_compartments("total_population", model_comps)
for s in age_strata:
    model.request_output_for_compartments(f"{s}_population", model_comps, strata={"age": str(s)})

In [38]:
model.run(parameters=params)
age_pops = model.get_derived_outputs_df()
age_pops.plot.area()

In [39]:
# Prepare calibration model
priors = [
    esp.UniformPrior("population growth rate", (0.005, 0.03)),
    esp.UniformPrior("starting population", (1e6, 3e7)),
]
targets = [est.NegativeBinomialTarget("total_population", target_pops, dispersion_param=100.0)]
bcm = BayesianCompartmentalModel(model, params, priors, targets)

In [40]:
# Set up optimisation
budget = 1000
opt_class = ng.optimizers.NGOpt
orunner = optimize_model(bcm, opt_class=opt_class, budget=budget)
start_params = {"population growth rate": 0.01, "starting population": 1.5e6}
orunner = optimize_model(bcm, opt_class=opt_class, suggested=start_params, init_method="midpoint")

In [41]:
# Optimise
rec = orunner.minimize(budget)
map_params = rec.value[1]
print("Best candidate parameters:")
for i_param, param in enumerate(map_params):
    print(f"\t{param}: {round(map_params[param], 3)} (within bound {priors[i_param].bounds()}")

Best candidate parameters:
	population growth rate: 0.011 (within bound (0.005, 0.03)
	starting population: 2124041.553 (within bound (1000000.0, 30000000.0)


In [42]:
# Run with optimised parameters
model.run(parameters=params | map_params)

In [43]:
# Inspect outputs
outputs = model.get_derived_outputs_df()
fig = go.Figure()
fig.add_trace(go.Scatter(x=outputs.index, y=outputs["total_population"], name="modelled"))
fig.add_trace(go.Scatter(x=target_pops.index, y=target_pops, name="target", mode="markers", marker=dict(color="black", size=2.0)))
fig.update_layout(yaxis={"range": [0.0, 3e8]})